<a href="https://colab.research.google.com/github/pablohenrique93/projeto-treino-etl-pandas-disney-movies-/blob/main/pandas_etl_project_disney_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação das Bibliotecas

In [ ]:
pip install pandera

In [ ]:
pip install pymongo

# Importação das Bibliotecas

In [3]:
import pandas as pd
import os
import numpy as np
import pymongo
from pymongo import MongoClient

In [4]:
# Importação das Bibliotecas para visualização de Dados
import matplotlib.pyplot as plt
import plotly.express as px

# Carregamento da Base de Dados

In [5]:
df = pd.read_csv('/content/disney_movies.csv')

In [ ]:
# Exibindo o Dataset
df

# Backup do Dataframe

In [7]:
# Faz-se necessário realizar um backup do dataframe carregado antes de qualquer manipulação de dados, em caso de algum erro
# nos processos posteriores, os dados estarão assegurados
backup_df = df.copy

# Pré Análise do Dataframe

In [ ]:
# Verificando o nome das colunas assim como os seus respectivos títulos
df.dtypes

In [ ]:
# Exibindo as 10 primeiras linhas do Dataframe
df.head(10)

In [ ]:
# Exibindo as 10 últimas linhas do Dataframe
df.tail(10)

In [ ]:
# Exibindo o tamanho do Dataframe
df.shape

# Tratamento da Base de Dados

In [ ]:
pd.unique(df['movie_title'])

In [ ]:
pd.unique(df['release_date'])

In [ ]:
pd.unique(df['genre'])

In [ ]:
pd.unique(df['mpaa_rating'])

In [ ]:
pd.unique(df['total_gross'])

In [ ]:
pd.unique(df['inflation_adjusted_gross'])

# Limpeza de Inconsistências e Padronização de Colunas

In [19]:
df.rename(columns={
    'movie_title':'titulo_do_filme',
    'release_date':'data_de_lancamento',
    'genre': 'genero',
    'mpaa_rating': 'mpa_classificacao',
    'total_gross': 'total_bruto',
    'inflation_adjusted_gross': 'bruto_ajustado_pela_inflacao'
},inplace=True)

In [ ]:
# Conferindo dataframe após manipulação
df.head()

In [22]:
# Renomeando e traduzindo registros da coluna 'genero'
df.replace({'genero':'Musical'}, 'musical',regex=True, inplace=True)
df.replace({'genero':'Adventure'}, 'aventura',regex=True, inplace=True)
df.replace({'genero':'Drama'}, 'drama',regex=True, inplace=True)
df.replace({'genero':'Comedy'}, 'comedia',regex=True, inplace=True)
df.replace({'genero':'Action'}, 'acao',regex=True, inplace=True)
df.replace({'genero':'Horror'}, 'terror',regex=True, inplace=True)
df.replace({'genero':'Thriller/Suspense'}, 'suspense',regex=True, inplace=True)
df.replace({'genero':'Western'}, 'ocidental',regex=True, inplace=True)
df.replace({'genero':'Documentary'}, 'documentario',regex=True, inplace=True)
df.replace({'genero':'Concert/Performance'}, 'concerto_performance',regex=True, inplace=True)

In [23]:
df['genero'] = df['genero'].str.strip()  # Remove espaços extras
df.replace({'genero': 'Romantic comedia'}, 'comedia_romantica', regex=True, inplace=True)

In [24]:
df['genero'] = df['genero'].str.strip()  # Remove espaços extras
df.replace({'genero': 'Black comedia'}, 'comedia_humor_negro', regex=True, inplace=True)

In [ ]:
# Conferindo dataframe após manipulação
pd.unique(df['genero'])

In [27]:
# Convertendo coluna "data_de_lancamento" para 'datetime'
df['data_de_lancamento'] = pd.to_datetime(df['data_de_lancamento'])

In [ ]:
# Conferindo após conversão
df.dtypes

In [ ]:
# Verificando se o dataframe possui dados faltantes, ou seja, nulls
df.isnull().sum()

In [30]:
# Como verificado na etapa anterior, notamos duas colunas com dados faltantes, desta forma iremos substituir por um valor personalizado
# Que neste caso será o termo "nao_definido"
valor_personalizado = "nao_definido"  # Substituição pelo valor desejado
df['genero'].fillna(valor_personalizado, inplace=True)
df['mpa_classificacao'].fillna(valor_personalizado, inplace=True)


In [ ]:
# Verificando o dataframe após alteração
df.isnull().sum()
# Como podemos notar, os valores nulos foram preenchidos, tornando o dataframe mais limpo

In [ ]:
# Verificando a existência de dados duplicados
duplicados = df.duplicated()
num_duplicados = duplicados.sum()
print(num_duplicados)
# Como podemos observar, não existe linhas duplicadas neste dataframe

# Visualização dos Dados

In [ ]:
# ----------------------------------- Top 5 Gêneros de Filmes Mais Frequentes -----------------------------------

# Contando quantos filmes de cada gênero existem usando 'value_counts()'
genero_counts = df['genero'].value_counts()

# Selecionando os 5 gêneros mais frequentes
top_generos = genero_counts.head(5)

# Criando um gráfico de barras
plt.figure(figsize=(10, 6))
top_generos.plot(kind='bar', color='skyblue')
plt.title('Top 5 Gêneros de Filmes Mais Frequentes')
plt.xlabel('Gênero')
plt.ylabel('Contagem')

# Exibindo o gráfico
plt.show()


In [ ]:
# ------------------------------- Distribuição da Classificação dos Filmes --------------------------------------

# Você pode contar quantos filmes de cada gênero existem usando 'value_counts()'
genero_counts = df['mpa_classificacao'].value_counts()

# Crie um gráfico de pizza
plt.figure(figsize=(8, 8))
plt.pie(genero_counts, labels=genero_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribuição da Classificação dos Filmes')
plt.axis('equal')  # Isso garante que o gráfico seja uma pizza perfeita

# Exiba o gráfico
plt.show()

In [ ]:
# ---------------------------------- Filmes com as Maiores Rendas e Suas Datas de Lançamento ----------------------------------

# Ordenação do DataFrame por renda em ordem decrescente e seleção dos 10 filmes com as maiores rendas
top_filmes = df.sort_values(by='total_bruto', ascending=False).head(10)

# Criando um gráfico interativo
fig = px.scatter(top_filmes, x='data_de_lancamento', y='total_bruto', text='titulo_do_filme',
                 title='Filmes com as Maiores Rendas e Suas Datas de Lançamento')

# Personalizando o layout do gráfico
fig.update_traces(textposition='top center', textfont_size=10)
fig.update_xaxes(title_text='Data de Lançamento')
fig.update_yaxes(title_text='Renda')

# Exibindo o gráfico
fig.show()

# Salvando Dataframe tratado localmente em formato CSV

In [45]:
df.to_csv('disney_movies_treated.csv', index=False)

# Envio para o MongoDB

In [40]:
# Criando conexão com o MongoDB

uri = "mongo_uri"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='mongo_key')

In [41]:
# Criando coleção para enviar para o MongoDB

db = client['disney_movies']
colecaotreated = db['disney_movies_treated']

In [ ]:
# Enviando o Dataset tratado para o MongoDB

df.reset_index(drop=True)
df01 = df.to_dict("records")
colecaotreated.insert_many(df01)

In [ ]:
# Verificando se o arquivo foi enviado com sucesso
colecaotreated.count_documents({})
# Como podemos conferir o dataset foi enviado corretamente para o MongoDB

# FIM DO PROCESSO